# Import python libraries

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from nltk import FreqDist, pos_tag
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfTransformer

# Import CountVectorizer and TFIDFVectorizer from feature_extraction.text.
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

### Import Train and Test Data

In [2]:
X_train= pd.read_csv("./datasets/X_train.csv" , squeeze = True)
y_train= pd.read_csv("./datasets/y_train.csv", squeeze = True)

X_test= pd.read_csv("./datasets/X_test.csv" , squeeze = True)
y_test= pd.read_csv("./datasets/y_test.csv", squeeze = True)

### Model 1: Naive Bayes : Pipeline - GridSearch

In [3]:
# Let's set a pipeline up with two stages:
# 1. CountVectorizer (transformer)
# 2. Multinomial Naive Bayes (estimator)
pipe = Pipeline([
    ('cvec', CountVectorizer()),
    ('tf', TfidfTransformer()),
    ('nb', MultinomialNB())
])

In [4]:
# Search over the following values of hyperparameters:
# Maximum number of features fit: 2000, 3000, 4000, 5000
# Minimum number of documents needed to include token: 2, 3
# Maximum number of documents needed to include token: 90%, 95%
# Check (individual tokens) and also check (individual tokens and 2-grams).
pipe_params = {
    'cvec__max_features' : [200, 300, 400, 500],
    'cvec__stop_words' : [None,['Sherlock' , 'Poirot'], 'english'],
    'cvec__ngram_range' : [(1,1), (1,2)],
    'nb__alpha':[1,3,5]
}

In [5]:
# Instantiate GridSearchCV.
gs_nb = GridSearchCV(pipe,
                      pipe_params,
                      cv = 5,
                      verbose=1,
                      n_jobs=8)

In [6]:
# Fit GridSearch to training data.
gs_nb.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 345 out of 360 | elapsed:    4.8s remaining:    0.2s
[Parallel(n_jobs=8)]: Done 360 out of 360 | elapsed:    4.9s finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('cvec', CountVectorizer()),
                                       ('tf', TfidfTransformer()),
                                       ('nb', MultinomialNB())]),
             n_jobs=8,
             param_grid={'cvec__max_features': [200, 300, 400, 500],
                         'cvec__ngram_range': [(1, 1), (1, 2)],
                         'cvec__stop_words': [None, ['Sherlock', 'Poirot'],
                                              'english'],
                         'nb__alpha': [1, 3, 5]},
             verbose=1)

In [7]:
gs_nb_df = pd.DataFrame(gs_nb.cv_results_)

In [8]:
gs_nb.best_estimator_

Pipeline(steps=[('cvec',
                 CountVectorizer(max_features=400, stop_words='english')),
                ('tf', TfidfTransformer()), ('nb', MultinomialNB(alpha=1))])

In [9]:
gs_nb.best_params_

{'cvec__max_features': 400,
 'cvec__ngram_range': (1, 1),
 'cvec__stop_words': 'english',
 'nb__alpha': 1}

### Run the model on Test data

In [10]:
# Evaluate the best fit model on the test data.
best_nb = gs_nb.best_estimator_
print(f"Training Score from best Naive Bayes: {best_nb.score(X_train , y_train)}")
print(f"Test Score from best Naive Bayes: {best_nb.score(X_test, y_test)}") 

Training Score from best Naive Bayes: 0.9603658536585366
Test Score from best Naive Bayes: 0.8893129770992366
